SCSD code

In [ ]:
from scsd import scsd
import numpy as np
import pandas as pd
from io import StringIO
from IPython.display import display, HTML

In [ ]:
!pip uninstall numpy==2.3.0 -y

In [ ]:
!pip install numpy==1.26.0

Calculating distortion based on NSD analysis for 50 Porphyrin samples in gas phase

In [ ]:
# imports your file - I find list compehensions easier than list.append()
molecule_name = "010"
with open(f"/Users/samirabaghbanbari/samira/FREQ-por/{molecule_name}.xyz") as f:
    lines = f.readlines()
    xyzmat = np.array([line.strip().split()[1:] 
                       for line in lines 
                       if line.startswith(("N", "C"))], dtype=float)
    
#this loads a model - the scsd model for porphyrin is the same as the NSD one
porphyrin_model = scsd.model_objs_dict.get("porphyrin")

#generates an scsd object, that contains our decomposition and plotting functions
scsd_mat = scsd.scsd_matrix(xyzmat, model = porphyrin_model)

#this runs the decomposition - the options slow the calculation but are necessary to avoid local minima
# and mis-assignment or mis-alignment, which would render the result meaningless
scsd_mat.calc_scsd(by_graph = True, bhopping = True)

#this simply displays the result. column 1 is the scsd value which is like an nsd value times 4. It's the
# sum of the movement of all atoms. (1) is the first normal mode, (2) is the second etc. 
#arr = scsd_mat.html_table(2)
print(f"molecule name is {molecule_name}")
display(HTML(scsd_mat.html_table(2)))

# Shows the nearest equivalent structures in the CCDC CSD
display(HTML(scsd_mat.compare_table()))

# this shows the "Mondrian" plot that gives an easy way to interpret the interactions of symmetry 
# and distortion
# the green square represents the C4v symmetry we'd expect from this type of molecule
%matplotlib notebook
scsd_mat.mondrian(cmap = "Spectral", as_type = "fig").show()

In [ ]:
%matplotlib inline
scsd_mat.mondrian(cmap = "Spectral", as_type = "fig").show()

In [ ]:

my_data = np.load("cropped_porphyrine.npy", allow_pickle=True)[:, :, 1:]
    
dfs_1, dfs_2 = [], []
porphyrin_model = scsd.model_objs_dict.get("porphyrin")
for index, xyzmat in enumerate(my_data, start=1):
    scsd_mat = scsd.scsd_matrix(xyzmat, model = porphyrin_model)
    scsd_mat.calc_scsd(by_graph = True, bhopping = True)
    print(f"molecule index is {index}")

    html_string_1 = scsd_mat.html_table(2)
    df_1 = pd.read_html(StringIO(html_string_1))[0]
    df_1["traj_id"] = index

    html_string_2 = scsd_mat.compare_table()
    df_2 = pd.read_html(StringIO(html_string_2))[0]
    df_2["traj_id"] = index

    dfs_1.append(df_1)
    dfs_2.append(df_2)

df1 = pd.concat(dfs_1)
df2 = pd.concat(dfs_2)

In [ ]:
df1.to_csv("scsd_50_porphyrine.csv")

In [ ]:
df_2